> IMPORTS


In [2]:
# Manipulação de Arquivos e Sistema
import os
import sys
from tqdm.notebook import tqdm # Versão especial do tqdm para o Jupyter

# Visão Computacional e Imagem
import cv2
import PIL.Image
from PIL import Image

# Machine Learning e Deep Learning (PyTorch)
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Biometria e Detecção Facial
import mediapipe as mp

# Processamento de Dados e Visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Verificação de Versões e Hardware
print(f"Versão do Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"MediaPipe: {mp.__version__}")

Versão do Python: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
PyTorch: 2.5.1
OpenCV: 4.10.0
MediaPipe: 0.10.31


> YOLO dataset training


In [3]:
from ultralytics import YOLO # Para detectar caminhões, prédios, etc.

model_yolo = YOLO('yolov8n.pt') # Modelo leve que identifica 80 tipos de objetos

def analisar_contexto(frame):
    results = model_yolo(frame)
    return results

> Teste de video teclado/mouse


In [4]:
import cv2
from ultralytics import YOLO

# Carrega o modelo 
model = YOLO('yolov8n.pt') 

# Caminho do vídeo
# Use o caminho exato que aparece no seu VS Code
video_path = 'video_test/teste_perifericos.mp4'

# Abre o vídeo
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo! Verifique o caminho.")

while cap.isOpened():
    success, frame = cap.read()
    
    if success:
        # Roda a detecção (stream=True é bom para vídeos longos)
        results = model(frame, stream=True)
        
        for r in results:
            # r.plot() cria a imagem com as caixinhas e nomes (mouse, keyboard)
            annotated_frame = r.plot() 
            
            # Mostra a janela
            cv2.imshow("IA Detectando Objetos Reais", annotated_frame)

        # O cv2.waitKey(1) é essencial para a janela não travar
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        # Fim do vídeo
        break

cap.release()
cv2.destroyAllWindows()


0: 640x384 1 keyboard, 108.8ms
Speed: 8.2ms preprocess, 108.8ms inference, 9.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 44.4ms
Speed: 2.8ms preprocess, 44.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 46.7ms
Speed: 2.6ms preprocess, 46.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 44.6ms
Speed: 2.8ms preprocess, 44.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 46.3ms
Speed: 2.3ms preprocess, 46.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 48.1ms
Speed: 8.4ms preprocess, 48.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 43.6ms
Speed: 2.1ms preprocess, 43.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 1 cell phone, 44.6ms
Speed: 2.0ms preprocess, 44.6ms inference, 0.7ms postproc

- Com a detecção feita com sucesso, esta na hora de tentar identificar anomalias em videos para detectar se é feito de IA ou não.


In [7]:
video_path = 'video_test/teste_perifericos.mp4'
cap = cv2.VideoCapture(video_path)

# Pega o total de frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
alertas_encontrados = [] # Lista para guardar as bizarrices detectadas

print(f"Iniciando análise de {total_frames} frames...")

while cap.isOpened():
    success, frame = cap.read()
    
    if not success:
        print("Fim do vídeo ou erro na leitura.")
        break

    # Roda a detecção
    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()
    
    # LÓGICA DE DETECÇÃO DE IA/ANOMALIAS
    analise_suspeita = False
    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            nome = model.names[int(box.cls[0])]
            
            # Se a confiança for baixa (comum em vídeos de IA instáveis)
            if conf < 0.50:
                analise_suspeita = True
                msg = f"Frame {frame_count}: Forma instável detectada ({nome})"
                if msg not in alertas_encontrados:
                    alertas_encontrados.append(msg)
                    print(f"{msg}")

    # Feedback Visual para o usuário
    status_cor = (0, 0, 255) if analise_suspeita else (0, 255, 0) # Vermelho se suspeito, Verde se ok
    status_txt = "ANALISE: SUSPEITA" if analise_suspeita else "ANALISE: REAL"
    
    cv2.putText(annotated_frame, status_txt, (50, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, status_cor, 3)
    
    # Contador de frames
    frame_count += 1
    cv2.putText(annotated_frame, f"Frame: {frame_count}/{total_frames}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Analisando Realismo", annotated_frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# LIMPEZA
cap.release()
cv2.destroyAllWindows()
for i in range(10):
    cv2.waitKey(1)

print("\n--- RESUMO DA ANALISE ---")
if alertas_encontrados:
    print(f"Foram detectadas {len(alertas_encontrados)} inconsistências no vídeo.")
else:
    print("Nenhuma inconsistência grave detectada. O vídeo parece real.")

Iniciando análise de 139 frames...
Frame 7: Forma instável detectada (cell phone)
Frame 8: Forma instável detectada (cell phone)
Frame 9: Forma instável detectada (cell phone)
Frame 10: Forma instável detectada (cell phone)
Frame 63: Forma instável detectada (keyboard)
Frame 79: Forma instável detectada (tv)
Frame 80: Forma instável detectada (tv)
Frame 80: Forma instável detectada (tie)
Frame 81: Forma instável detectada (tie)
Frame 81: Forma instável detectada (person)
Frame 82: Forma instável detectada (cell phone)
Frame 82: Forma instável detectada (person)
Frame 82: Forma instável detectada (tie)
Frame 83: Forma instável detectada (tie)
Frame 93: Forma instável detectada (cell phone)
Frame 94: Forma instável detectada (laptop)
Frame 100: Forma instável detectada (mouse)
Frame 101: Forma instável detectada (mouse)
Frame 102: Forma instável detectada (mouse)
Frame 106: Forma instável detectada (remote)
Frame 113: Forma instável detectada (remote)
Frame 114: Forma instável detectada 